# Introducción

- Se tiene como objetivo inicial de este proyecto superar la cifra de la predicción ganadora de [una competición de Kaggle ya cerrada](https://www.kaggle.com/competitions/diamonds-part-datamad0122/) y ser capaz de predecir el precio de cualquier diamante a partir de sus características

- Se usa un segundo en lugar del de la competición para obtener los precios a partir de esas características. Ese otro modelo se entrena con [el "dataset" original de diamantes](https://www.kaggle.com/datasets/swatikhedekar/price-prediction-of-diamond), pues su variable "target" es más precisa que la del torneo, dado que no se ha escalado ni redondeado

- El objetivo secundario es predecir el precio a partir de una imagen. Esto se hace con una combinación de modelos. [El "dataset" con las imágenes de los diamantes y sus características se obtiene, también, de Kaggle](https://www.kaggle.com/datasets/harshitlakhani/natural-diamonds-prices-images)

- Con los modelos que obtienen el precio a partir de una foto y el que predice los precios a partir de sus características se crea, además, [una "app" de Streamlit que les saca partido](https://rogerperello-machine-learning-diamon--streamlitfilesmain-bke5k8.streamlit.app/)

- La fuente del "dataframe" de competición y el original, según señala el autor en Kaggle, es esta: "[Tiffany & Co's snapshot pricelist from 2017](https://www.chegg.com/homework-help/questions-and-answers/data-source-tiffany-co-s-snapshot-price-list-2017-data-detail-ofiginal-dataset-peblishod-t-q103429046)" 

- En cuanto al autor de las imágenes de los diamantes, también en Kaggle, [las ha obtenido con "webscrapping"](https://capitalwholesalediamonds.com/)


# EDA

- Es el análisis exploratorio de los datos de competición. Como las variables son las mismas, los cambios detectados posibles se extrapolan para el "dataset" original

- Se lleva a cabo en el "notebook" homónimo


## Observaciones iniciales

- Se adaptan los nombres de las columnas para que sean legibles

- No se detectan nulos

- Se encuentran duplicados, pero como todos tienen su propia identificación se asume que cada diamante es único. Por tanto, se conservan

- Se detecta que hay diamantes con 0 milimetros de altura, anchura y/o profundidad

## Cambios probados

- Se detectan una serie de características:

1) Varias columnas tienen unos pocos valores atípicos extremadamente altos ["depth (percentage)", "table (percentage)", "width (millimeters)", "depth (millimeters)"]

2) Como se ha anticipado, varias filas tienen un cero en todas las variables relacionadas con el tamaño de los diamantes, sin contar el peso ["length (millimeters)", "width (millimeters)" y "depth (millimeters)"].

3) Dos columnas con una alta correlación negativa tienen unos pocos "outliers" que destacan moderadamente y que, además, son compartidos ["depth (percentage)" y "table (percentage)"].

4) Si se borrasen las filas con ceros del punto 2, todavía quedaría un cero en una fila para el "length". Como los diamantes son prácticamente circulares, "length" y "width" son casi idénticas ["length (millimeters)"].

5) Si se borrasen las filas con ceros del punto 2 y se solucionase el cero del punto 4, todavía quedaría un cero en una fila para "depth (millimeters)". Se descubre que la variable "depth (percentage)" se obtiene de dividir el "depth (millimeters)" correspondiente entre la media de "width" y "length" ["depth (millimeters)"].

6) Se detecta un "outlier" en "lenght" ["length (millimeters)"].

7) Varias columnas son asimétricas por la derecha. Con el logaritmo, se centrarían esas distribuciones y deaparecerían "outliers" ["weight (carat)", "length (millimeters)", "width (millimeters)" y "depth (millimeters)"].

8) Existe un valor atípico en "weight". Sin embargo, imputarlo al valor máximo de su "boxplot" haría que resurgiese después de un escalado estándar ["weight (carat)"].

9) Dos columnas tienen una alta correlación negativa que se quiere conservar, y muchos "outliers" no compartidos. Se presume que la posición de esos "outliers" es la que determina la correlación ["depth (percentage)" y "table (percentage)"].

10) "depth (millimeters)" tiene valores atípicos. Esta variable, junto con las demás relacionadas con el tamaño, están tan correlacionadas que casi podría considerarse que son casi la misma variable escalada ["depth (millimeters)"].

11) Algunas variables tienen valores mucho más altos que el resto.

----------------------------------

- Con esa información, se prueban una serie de posibles cambios a llevar a cabo durante la fase de "feature_engineering". Si se aplica el conjunto de estos cambios, el "dataframe" queda libre de nulos completamente:

1) Borrado de "outliers" extremadamente altos ["depth (percentage)", "table (percentage)", "width (millimeters)", "depth (millimeters)"].

2) Borrado de filas que tienen 0 en todas las variables relacionadas con el tamaño excepto el peso ["length (millimeters)", "width (millimeters)" y "depth (millimeters)"].

3) Borrado de los "outliers" compartidos moderadamente altos ["depth (percentage)" y "table (percentage)"].

4) Asignación del valor con 0 restante en "length" al "width" correspondiente ["length (millimeters)"].

5) Asignación del valor con 0 restante de "depth (millimeters)" a partir de una operación con el "length", el "width" y el "depth (percentage)" correspondientes ["depth (millimeters)"].

6) Asignación del "outlier" restante del "length" al "width" correspondiente ["length (millimeters)"].

7) Transformación con logaritmo ["weight (carat)", "length (millimeters)", "width (millimeters)" y "depth (millimeters)"].

8) Imputación al siguiente valor más alto ["weight (carat)"].

9) Imputación a los valores máximos y mínimos del "boxplot" ["depth (percentage)" y "table (percentage)"].

10) Neutralización de "outliers" con un modelo "ridge" ["depth (millimeters)"].

11) Escalado.

## Cambios no probados

- Se detectan una serie de características:

12) La variable "depth (percentage)" sale de dividir "depth (millimeters)" por la media de "length (millimeters)" y "width (millimeters)". Por tanto, si se hace la operación, el resultado debería coincidir con la columna de "depth (percentage)". Sin embargo, no es así. Habrá que probar durante el "feature engineering" si utilizar los valores calculados mejora la predicción.

13) Las columnas relacionadas con el tamaño tienen una correlación altísima. Quizá los resultados mejoren con borrarlas. Otras columnas tinen una correlación ínfima, cercana a 0, que podría considerarse irrelevante

14) Las filas con el valor máximo de "clarity quality" (7, variable discreta) no cambian en cuanto a las variables relacionadas con el tamaño si se comparan con las que tienen una "clarity quality" un punto por debajo (6). Quizá cabría imputar esos valores 7 de "clarity quality" al 6.


# Feature_engineering

- Se aplican los cambios probados y no probados durante el EDA intercalando múltiples modelos para ver 1) qué modelos mejoran con qué cambios, y 2) qué modelo es el adecuado para la optimización. Se trabaja con el "dataset" de competición, pero como ya se ha anunciado, los resultados son extrapolables al original

- Se lleva a cabo en el "notebook" homónimo

## Selección de cambios

- Se reúnen los cambios apuntados en el EDA en subgrupos:

a) Borrado: 1, 2 y 3 (borrado parcial: solo 2)

b) Asignación: 4, 5, 6 (asignación parcial: solo 4 y 5)

c) Logaritmo: 7

d) Valor más alto de "weight": 8

e) Imputaciones "boxplot": 9

f) Imputaciones "ridge": 10

g) Escalado "MinMax": 11

h) Escalad "Standard": 11

i) Sustitución: 12

j) Descarte correlación altísima: 13 -> ningún modelo mejora

j) Descarte correlación ínfima: 13

k) Imputaciones "clarity quality": 14

--------------------------------

- Cada uno de los modelos mejora con los siguientes cambios:


---------- LinearRegression ----------

· Borrado

· Asignación

· Logaritmo

· Imputaciones "boxplot"

· Imputaciones "ridge"

· Sustitución

· Imputaciones "clarity quality"

---------- Ridge ----------

· Escalado "MinMax"

· Borrado

· Asignación

· Logaritmo

· Imputaciones "boxplot"

· Imputaciones "ridge"

· Sustitución

· Imputaciones "clarity quality"

---------- KNeighborsRegressor ----------

· Escalado "Standard"

· Borrado parcial

· Asignación

· Valor más alto de "weight"

· Imputaciones "boxplot"

· Imputaciones "ridge"

· Sustitución

· Descarte correlación ínfima

· Imputaciones "clarity quality"

---------- SVR ----------

· Escalado "Standard"

· Borrado

· Asignación

· Logaritmo

· Valor más alto de "weight"

· Imputaciones "boxplot"

· Imputaciones "ridge"

· Sustitución

· Descarte correlación ínfima

---------- DecisionTree ----------

· Borrado parcial

· Asignación parcial

---------- RandomForest ----------

· Borrado parcial

· Asignación parcial

· Sustitución

---------- XGBRegressor ----------

· Borrado parcial

· Asignación parcial


## Selección de modelos

- Estos son los resultados, una vez aplicados los cambios y tuneados algunos hiperparámetros, si se compara la predicción con el "y_test":

a) "LinearRegression" mejora su "rmse" de 0.182664 a 0.143851 (-21.24%)

b) "Ridge" mejora su "rmse" de 0.182826 a 0.144234 (-21.11%)

c) k vecinos mejora su "rmse" de 0.181632 a 0.121120 (-33.32%)

d) "SVR" mejora su "rmse" de 0.189328 a 0.100319 (-47.01%)

e) "DecisionTree" mejora su "rmse" de 0.130480 a 0.109705 (-15.92%)

f) "RandomForest" mejora su "rmse" de 0.094246 a 0.091972 (-2.41%)

g) "XGBRegressor" mejora su "rmse" de 0.090762 a 0.086893 (-4.26%)

- Se elige trabajar en la siguiente fase con "XGB". Es el mejor con una "rmse" de 0.086893, y, como dispone de muchos más hiperparámetros que los probados hasta ahora, probablemente tenga un margen considerable de optimización

- El orden, si "XGB" (1) no cumpliese o fuera necesario otro modelo, sería: 2) "RandomForest"; 3) "SVR"; 4) "DecisionTree"; 5) k vecinos; 6) regresión lineal; y 7) "ridge"

# Target_transformation

- Se confirma que la variable "target" del "dataframe" de competición se ha escalado, y no representa los precios reales de los diamantes

- En el "notebook" homónimo se investiga si es cierto que los dos "datasets" son la misma cosa y qué tipo de escalado se ha usado

- El escalado efectuado es el logaritmo. Sin embargo, se han perdido decimales en el proceso. Por tanto, hay que usar el "dataframe" de competición para competir, y el original para predecir precios reales con mayor concreción


# Images_data_processing

- En el "notebook" homónimo, se tratan varios "datasets" a parte que representan un conjunto de características de imágenes de diamantes. 

- El proceso es el siguiente:

1) Se unen los "datasets" en un solo "dataframe". Este es el "dataframe" que se utilizará para predecir el precio a partir de las imágenes

2) Se copia el "dataframe" y, mediante diversas técnicas, se adapta el para que contenga las mismas variables que el ya visto en el EDA. Esto se hace con fines meramente exploratorios, y no se detecta nada que impida la predicción del precio a partir de la imagen

- Durante el proceso exploratorio se detecta un mismo diamante con dos precios, así que se elimina. Además, se borran los duplicados

- Por otro lado, se elimina una coma y se pasa la variable "target", el precio, a numérica

# Model_optimization_price

- Donde se experimenta de forma extensiva con los modelos necesarios con tal de obtener los mejores resultados posibles

- El primer modelo entrenado ("XGB") que sale es el que se usa para la competición

- El segundo (también "xgb") y el tercero (k vecinos), entrenados con el "dataframe" original de diamantes, no el de competición, se usan mediante un "stacking" para predecir los precios reales de los diamantes a partir de sus características. Se usan dos porque "xgb" por si solo es incapaz de dar resultados correctos si se le pasan, para la predicción, valores por encima o por debajo de los máximos o mínimos de las variables vistas durante el entrenamiento. Eso es un problema recurrente con los modelos de árboles, y "SVR" es muy lento como alternativa, así que se usa k vecinos

- Así, se hace un "stacking" de "XGB" y k vecinos para obtener un cuarto modelo, el bueno, para predecir los precios en la "app". Ese modelo combina la solidez de "XGB" y la flexibilidad de k vecinos para predecir con valores por encima o por debajo de los máximos y mínimos de las variables vistas durante el entrenamiento

- El proceso se lleva a cabo en el "notebook" homónimo

## Mejoras para competición

- Estos son los hiperparámetros que se añaden al modelo a cada ronda:

1) Se prueba sin modificación alguna para tener una base desde la que hacer comparaciones

2) Se prueba con los cambios de "feature engineering" (borrado parcial y asignación parcial) por la misma razón

3) Se obtiene un número de estimadores óptimo como punto de partida y se tunea el "learning rate" ("eta"). Una vez tuneado, se vuelve a buscar el número óptimo de estimadores

4) Se aplican "constaints" a la columna "weight", ya que solo crece con el precio

5) Se establecen un "subsample" y "colsample_bytree" de 0.8, que es el estándar recomendable, y se tunean "max_depth" y "min_child_weight"

6) Se tunea "gamma", parámetro de semiregularización

7) Se prueban dos opciones y se tunean: a) "sampling" combinado con "colsample_bytree", y b) "sampling_method='gradient_based" combinado con "colsample_bytree". Gana la opción "a"

8) Se tunean los hiperparámetros de regularización ("alpha", "lambda" y "max_delta_step"), que son los que ayudan a prevenir el "overfitting" y el "underfitting"

9) Se aumenta el número de árboles en paralelo, convirtiendo, a todos los efectos, el "xgboost" en un "random forest" con "booster"

10) Se reajusta el número de estimadores una vez más

### Resultado competición

- Se superan los puntos del ganador de la competición, con una "rmse" de 0.08506 contra 0.08617

- Como el modelo predice bien con datos que no ha visto, hace pensar que no existe "overfitting"

## Mejoras para visualización A

- Se repite el mismo proceso que en el punto anterior, pero esta vez se usa el original en lugar del de competición

- Se usa el "dataset" original porque los precios son más precisos. Se le aplica el logaritmo a la "target" para facilitar el proceso, que es así como venía el de competición, pero se conservan todos los decimales

### Resultado A

-  Es mejor que el del anterior modelo (normal, ya que se han usado los precios reales)

- Al probar con pasarle diferentes combinaciones de valores, se detecta que este modelo no predice bien cuando recibe cifras que superan o que son inferiores a los máximos y mínimos de las variables vistas durante el entrenamiento. Eso ocurre porque los modelos de árboles no son tan precisos en los extremos, y, como se ha notado en "feature_enginerring" y en el EDA, la "r2" es alta, y hay varias variables con una correlación altísima. Por eso, se crea un modelo B de predicción de precios

- Aunque pudiera parecerlo, eso no ocurre porque haya "overfitting", pues el modelo "xgboost" usado para la competición ha predicho a partir de datos desconocidos estupendamente

## Mejoras para visualización B

- Con tal de predecir el precio de diamantes cuyas características sean superiores o inferiores a los máximos o mínimos de las variables vistas durante el entrenamiento, se elige el siguiente mejor modelo no "de árboles" detectado durante en "feature_engineering": "SVR"

- Sin embargo, como su coste computacional es tan elevado, y se presume que será a aún mayor si se lleva a cabo un "stacking", se da preferencia el siguiente de la lista: k vecinos

- A pesar de lo visto en el "feature_engineering", como se pretende aprovechar el modelo para valores muy altos o bajos a la hora de predecir, interesa entrenarlo con "outliers". Por tanto, no se le hace más tratamiento que el que ha recibido "xgboost"

### Resultado B

- La predicción con el "y_test" es peor, lo que confirma que "xgboost" trabaja mejor que k vecinos. Sin embargo, es un buen complemento para predecir si los valores que se le pasan están fuera de los máximos y mínimos de las variables vistas en el entrenamiento

## "Stacking"

- Se combinan las bondades de ambos modelos con un "stacking". Para ello se usa un tercer modelo sencillo de regresión lineal

- El modelo resultante predice tan bien como "xboost" y, además, es capaz de predecir a partir de valores superiores o inferiores a los máximos y mínimos de las variables vistas durante el entrenamiento

- Así, este es el modelo que se usa para la predicción de precios en la "app"

# Inflation_analysis

- [Se utiliza "webscrapping"](https://www.in2013dollars.com/Jewelry/price-inflation/2017-to-2023?amount=326) para obtener automáticamente la cifra del aumento por inflación respecto a 2017, fecha en que se compuso el "dataframe" original de precios

- En 2023, respecto a 2017, los precios han subido cerca de un 11%

- Se implementa este método en la "app", tanto para la predicción a partir de características, cuyo "dataset" es de 2017, como la de imágenes, de 2022

- Se detecta también una alternativa para aproximar la inflación mediante la media de los últimos años, en caso de que la página dejara de funcionar puntualmente

# Images_processing

- En el "notebook" homónimo se iguala el contenido de la carpeta de imágenes procesadas a las "Id" de diamantes que contiene el "dataframe" correspondiente. Esto se hace porque no todas las imágenes son buenas, las hay que están en blanco. Además, se han borrado duplicados en "images_data_processing"

- Además, se hace una pequeña exploración de las imágenes con una PCA para ver el mínimo número de componentes y que aspecto tienen

- Se determina que, para el modelaje con red neuronal, se necesitará un modelo que acepte un "input" RGB con más de 150 componentes

- Tas trabajar un poco con el siguiente "notebook" (el de "Model_optimization_price") se descubre que la predicción es terrible. Por tanto, se regresa a este y se borran los "outliers" de la variable "target", lo que mejora considerablemente los resultados

# Model_optimization_image

- En el "notebook" homónimo, se procura obtener un modelo que pueda predecir el precio de los diamantes a partir de sus imágenes

- Al final, la predicción se acaba haciendo a partir de las imágenes y del peso de los diamantes, ya que tiene una gran influencia y el modelo inicial no parece capaz de dar buenos resultados

## Modelo provisional: MobilenetV3Large

- Se prueban muchos modelos (VGG16, Resnet50...) con sus diferentes versiones, pero no se guardan porque el código es largo y complicaría innecesariamente la revisión del "notebook". La arquitectura y el funcionamiento es similar, solo cambia el tamaño del "input"

- Al final, se utiliza un modelo MobileNetV3Large, que es potente, relativamente moderno, de una fuente confiable (Google), y que se ha demostrado eficaz para la detección de imágenes

- Como es muy lento, se pone en marcha en Google Collab y se aprovecha la GPU

- Como la predicción no es muy buena, se considera provisional. Un segundo modelo usará esa predicción junto con el peso de los diamantes para obtener un precio más ajustado a la realidad

# Modelo definitivo: SVR

- Se decide crear un modelo de "machine learning" que trate de mejorar la predicción del precio de los diamantes a partir del peso y de la predicción provisional hecha anteriormente por la red neuronal. Se hace así por ser más transparentes que las redes neuronales, lo que permite elegir uno u otro en función del comportamiento de los datos más fácilmente

- Se hace un "baseline" de modelos supervisados, se encuentra que SVR es el más adecuado por tener buenas métricas y poca variación, y se optimiza con el "kernel" lineal para aproximar el aumento del precio de los diamantes al subir su peso, así como para tener en cuenta que la predicción final debería crecer a medida que la provisional aumenta

- Si bien el precio de los diamantes en relación a su peso es exponencial, eso solo es determinante cuando el peso de los diamantes es elevado. Para diamantes pequeños, puede considerarse que es casi lineal; de ahí que se seleccione SVR con ese "kernel" (los diamantes de los que se dispone no son muy grandes en su mayoría)


# Sobre la "app"

- La mayoría de los detalles de su funcionamiento ya se han dado en los puntos anteriores o pueden verse en el desplegable "a peek into the process" dentro de la propia "app"

- La predicción de diamantes de peso mayor 2 quilates se complica (tanto si se hace respecto a las imágenes como con las características). Eso se debe a que los datos de los que se dispone tienen pocos diamantes grandes y, además, no consideran algunas características (como la simetría o la fluorescencia) que, si bien pueden obviarse en los diamantes pequeños, su importancia crece considerablemente con los mayores

- Por esa razón, a la hora de hacer la "app" se limita la introducción de quilates a 2. Eso impide que las predicciones sean desastrosas y, al mismo tiempo, permite seguir teniendo en cuenta la mayoría de diamantes que hay en el mercado, que son más bien pequeños

- Con todo lo hecho, se crea la "app" en Streamlit. Puede verse [aquí](https://rogerperello-machine-learning-diamon--streamlitfilesmain-bke5k8.streamlit.app/)

# Consideraciones finales

## ¿Que problema hay que resolver?

- Evaluar el precio de un diamante no es cosa fácil, y cuensta entre 50 y 150 dólares si se deja en manos de un profesional. Si el diamante es pequeño, eso puede ser un porcentaje importante de su valor

## ¿Que solución se aporta?

- Se ofrece una aproximación del precio de un diamante a partir de sus características o de su imagen, siempre y cuando este no supere los 2 quilates. Para los que son más grandes es recomendable acudir a un profesional; como el precio del diamante será mayor, saldrá a cuenta el servicio

## ¿Qué modelos se han probado?

- Muchísimos. Para predecir el precio a partir de las características:

1) Se ha hecho un baseline con "LinearRegression", "Ridge", k vecinos, "SVR", "DecisionTree", "RandomForest" y "XGBRegressor"

2) Se ha hecho un "stacking" con los modelos tuneados de "XGBRegressor" y k vecinos, unidos por una regresión lineal

Para predecir el precio a partir de las imágenes:

1) Se han probado varios modelos de "transfer learning" para obtener un precio provisional (VGG16, Resnet50, mobilenet...), así como varias versiones de los mismos. Se ha elegido, al final, MobilenetV3Large

2) Se han probado varios modelos de "machine learning" para mejorar esa predicción al juntarla con el peso de los diamantes en quilates. El baseline ha incluido "LinearRegression", "Ridge", k vecinos, "SVR", "DecisionTree", "RandomForest" y "XGBRegressor". Se ha elegido "SVR" por su buen rendimiento y se ha tuneado con el "kernel" lineal, entre otros hiperparámetros, por razones ya explicadas

## ¿Qué resultados y conclusiones se han obtenido?

- Es muy complicado obtener una predicción del precio de un diamante, pues hay muchos factores que no pueden tenerse en cuenta, como la moda actual (por ejemplo, es posible que un determinado color suba puntualmente de precio) o el criterio del vendedor. Además, los datos de los que se dispone son limitados, y no incluyen factores como la fluorescencia o la simetría. Con todo, es factible obtener una predicción que no sea escandalosa para diamantes que no sean exageradamente grandes

## ¿Cuáles han sido las variables de mayor impacto?

- Con diferencia, el principal factor que determina el precio de un diamante es el peso

## ¿Qué consecuencias tiene en términos de negocio?

- La predicción obtenida probablemente no sirva para tasar directamente el diamante, pero sí para determinar, en función del criterio del usuario, si supera un varemo que justifique su venta, o para corroborar una estimación previa

## ¿Cómo podría mejorarse?

- Sin recurrir a Kaggle, ignorando la competición, haciendo el "webscrapping" directamente para obtener fotos y características. Se elegiría una página como [bluenile](https://www.bluenile.com/es/en/diamond-details/LD21034945?track=FCOM&slot=1&type=1), donde los diamantes pueden observarse en diferentes perspectivas, para poder evaluar también la profundidad y detectar más imperfecciones. Además, se extraerían características de las que actualmente no se dispone, como la simetría y el "culet"

- Obteniendo una muestra mayor, que incluyese más diamantes con precios altos